# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'new_transport_interface']

## Transport layer setup

Create the Transport layer and run it in the background.

In [3]:
tiface_inner = await new_transport_interface("0.0.0.0", 0)
tiface_inner.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f7bac5f6970>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [4]:
class LoggingTransportInterface:
    def __init__(self, inner):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        
    async def command_response(self, transport_cmd):
        print(f"LTI{self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        print(f"LTI{self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            print(f"LTI{self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
tiface = LoggingTransportInterface(tiface_inner)
tiface

# Filter layer setup

In [5]:
fiface = FilterInterface(tiface, FilterModeW("client"))

Find methods to run

In [6]:
[m for m in dir(fiface) if not m.startswith('__')]

['command_response', 'get_notifications', 'notif_poll_ms', 'run']

Run Filter!

In [7]:
run_fut = fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not run_fut.done()

LTI0: got transport notifications: []


Get the number of milliseconds between calling `get_notifications` on the contained `TransportInterface`. This can also be assigned to at any time. Lower values can cause performance issues, while higher values add latency for anything involving filter notifications.

In [8]:
fiface.notif_poll_ms

30

In [9]:
fiface.notif_poll_ms = 100

## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient4222", client_version: "0.0.666" }

In [11]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=chococat_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(157.230.134.224:2016), action: Connect { name: "FilterNotebookClient4222", client_version: "0.0.666" } }

In [12]:
await fiface.command_response(filter_cmd)

LTI5 skipped 4: received transport_cmd NewEndpoint { endpoint: Endpoint(157.230.134.224:2016), timeout: 250ms }
LTI6: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI7: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient4222", client_version: "0.0.666" } }] }
LTI8: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI9: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Response { sequence: 0, request_ack: Some(0), code: LoggedIn { cookie: "DCmCJlp2yP84WyQP", server_version: "0.3.5" } } }]


Accepted

In [13]:
fiface.get_notifications()

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: LoggedIn { cookie: "DCmCJlp2yP84WyQP", server_version: "0.3.5" } }]

In [14]:
new_response_code = _[0]
new_response_code

NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: LoggedIn { cookie: "DCmCJlp2yP84WyQP", server_version: "0.3.5" } }

In [15]:
cookie = new_response_code.response_code.cookie
cookie

'DCmCJlp2yP84WyQP'

### Convenience function

In [16]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await fiface.command_response(filter_cmd)

### Join a room

In [17]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

LTI10: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(0), cookie: Some("DCmCJlp2yP84WyQP"), action: JoinRoom { room_name: "general" } }] }
LTI11: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI12: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(157.230.134.224:2016) }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Response { sequence: 1, request_ack: Some(1), code: JoinedRoom { room_name: "general" } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong 

Accepted

LTI13: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 250ms }], packets: [Request { sequence: 3, response_ack: Some(0), cookie: Some("DCmCJlp2yP84WyQP"), action: KeepAlive { latest_response_ack: 0 } }] }


Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [18]:
fiface.get_notifications()

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: JoinedRoom { room_name: "general" } },
 NewChats { endpoint: Endpoint(157.230.134.224:2016), messages: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }] }]

In [21]:
fiface.get_notifications()

LTI2776: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 6260007738958029240 } } }, TransportNotice::EndpointIdle { endpoint: Endpoint(157.230.134.224:2016) }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 13482629177316894215 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebook

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: BadRequest { error_msg: "cannot join game because in-game" } }]

LTI2777: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 8598715098652477373 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 13830203448992411633 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_

LTI2779: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 9055349769036180774 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 13959361754886622046 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_

LTI2781: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 9954461476103735198 } } }, TransportNotice::EndpointIdle { endpoint: Endpoint(157.230.134.224:2016) }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 17779245231655907842 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebook

LTI2783: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 10294833991438285 } } }, TransportNotice::EndpointIdle { endpoint: Endpoint(157.230.134.224:2016) }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 11606090810505006053 } } }, TransportNotice::EndpointIdle { endpoint: Endpoint(157.230.134.224:2016) }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { 

LTI2785: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 7453564582438132336 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 12450038515134482432 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_

LTI2787: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 860371146015963713 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 10603590503228692860 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_u

LTI2789: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 13183091701640724057 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 8817623401282210901 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(17), player_name: "Server", message: "Player FilterNotebookClient9548 has left." }], game_update_seq: None, game_updates: [], universe_

In [23]:
fiface.get_notifications()

[]